In [1]:
import pandas as pd
from tqdm import tqdm

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

In [ ]:
# new_folder = path+str(routeNo)
# os.makedirs(new_folder)

In [2]:
ospath = '/Users/nguyenduykhang/Documents/Projects/HPCC-BusAlgos/'
path_route_df = ospath + 'data/all_routes.csv'

all_routes = pd.read_csv(path_route_df)
num_routes = len(all_routes)

*Tong hop thong tin tat ca cac tram dung [StopID, 'Code, 'Name', 'StopType', 'Zone', 'Street', 'Lng', 'Lat', 'Routes']*

thời gian di chuyền trung bình từ hub này đến hub khác theo các interval, trong các khoảng interval thì tìm được hub trung gian để có thời gian di chuyển nhỏ nhất, tương đương với ma trân P (có hay không một chuyến bus giữa 2 Hub đang xem xét)

--> ma trận thời gian di chuyển trung bình giữa 2 Hub theo các interval trong ngày
==> với trong từng interval, di chuyển giữa 2 Hub thì đi qua Hub trung gian nào là tối ưu nhất.
tuy nhiên, nếu start_time ngay gần với giao của 2 interval (vd, 10h55) thì cái Hub trung gian trong cái interval hiện tại có thể không là tối ưu nhất

--> tạo dự liệu giả để thử nghiệm

In [4]:
# Tong hop thong tin cua tat ca cac tram bus

# all_bus_stops = list()
all_bus_stops_ID = list()

ospath_stop = '/Users/nguyenduykhang/Documents/Projects/HCMC-Bus-Network/data/buyttphcm'

stops_data = list()
for route in tqdm(range(num_routes)):
    routeNo = all_routes.iloc[route]['RouteNo']
    for dir_flag in [0, 1]:
        if dir_flag == 0:
            stops = pd.read_csv(f"{ospath_stop}/{routeNo}/stops_by_var.csv")
        else:
            stops = pd.read_csv(f"{ospath_stop}/{routeNo}/rev_stops_by_var.csv")
        
        num_stops = len(stops)
        for stop in range(num_stops):
            StopId = stops.iloc[stop]['StopId']
            if StopId not in all_bus_stops_ID:
                all_bus_stops_ID.append(StopId)
                row = [stops.iloc[stop]['StopId'],
                    stops.iloc[stop]['Code'],
                    stops.iloc[stop]['Name'],
                    stops.iloc[stop]['StopType'],
                    stops.iloc[stop]['Zone'],
                    stops.iloc[stop]['Street'],
                    stops.iloc[stop]['Lng'],
                    stops.iloc[stop]['Lat'],
                    stops.iloc[stop]['Routes']]
                stops_data.append(row)
                
all_bus_stops = pd.DataFrame(stops_data, columns=['StopId', 'Code', 'Name', 'StopType', 'Zone', 'Street', 'Lng', 'Lat', 'Routes'])

all_bus_stops.info()

  0%|          | 0/128 [00:00<?, ?it/s]

100%|██████████| 128/128 [02:25<00:00,  1.14s/it]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4414 entries, 0 to 4413
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   StopId    4414 non-null   int64  
 1   Code      4414 non-null   object 
 2   Name      4414 non-null   object 
 3   StopType  4414 non-null   object 
 4   Zone      4414 non-null   object 
 5   Street    4414 non-null   object 
 6   Lng       4414 non-null   float64
 7   Lat       4414 non-null   float64
 8   Routes    4414 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 310.5+ KB


In [5]:
all_bus_stops.to_csv(f"{ospath}data/hcmc_bus_stops_information.csv", index=False)

*Tong hop thong tin cac Hub nodes [HubId, StopId, Route]*

In [19]:
# Tong hop danh sach Hub, sort lai theo thu tu id
hub_nodes_StopId = list()

for route in tqdm(range(num_routes)):
    routeNo = all_routes.iloc[route]['RouteNo']
    for dir_flag in [0, 1]:
        hub_nodes = pd.read_csv(f"{ospath}data/bus-lines/{routeNo}/{dir_flag}_hub_nodes.csv")
        hubs = hub_nodes['StopId']
        hub_nodes_StopId = list(set(hubs).union(set(hub_nodes_StopId)))

  0%|          | 0/128 [00:00<?, ?it/s]

100%|██████████| 128/128 [00:00<00:00, 380.37it/s]


In [20]:
all_hub_nodes = all_bus_stops[all_bus_stops.StopId.isin(hub_nodes_StopId)]
num_hub_nodes = len(all_hub_nodes)
all_hub_nodes.drop(columns=['Code', 'Name', 'StopType', 'Zone', 'Street'])
all_hub_nodes.sort_values(by=['StopId'], inplace=True)
all_hub_nodes.insert(0, 'HubId', range(num_hub_nodes))

all_hub_nodes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 28 to 543
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   HubId     1005 non-null   int64  
 1   StopId    1005 non-null   int64  
 2   Code      1005 non-null   object 
 3   Name      1005 non-null   object 
 4   StopType  1005 non-null   object 
 5   Zone      1005 non-null   object 
 6   Street    1005 non-null   object 
 7   Lng       1005 non-null   float64
 8   Lat       1005 non-null   float64
 9   Routes    1005 non-null   object 
dtypes: float64(2), int64(2), object(6)
memory usage: 86.4+ KB


In [22]:
all_hub_nodes.to_csv(f"{ospath}data/hub_nodes_information.csv", index=False)

In [21]:
list(all_hub_nodes.StopId)

[1,
 2,
 3,
 4,
 5,
 8,
 11,
 14,
 18,
 25,
 26,
 27,
 31,
 32,
 33,
 34,
 35,
 43,
 47,
 50,
 56,
 57,
 60,
 72,
 75,
 76,
 79,
 85,
 86,
 89,
 90,
 95,
 97,
 100,
 103,
 110,
 111,
 114,
 115,
 117,
 120,
 121,
 122,
 126,
 137,
 138,
 140,
 141,
 142,
 144,
 153,
 154,
 155,
 166,
 169,
 173,
 174,
 175,
 177,
 184,
 186,
 187,
 192,
 195,
 199,
 200,
 201,
 208,
 212,
 213,
 219,
 221,
 223,
 224,
 225,
 229,
 232,
 233,
 234,
 237,
 242,
 244,
 246,
 252,
 256,
 258,
 259,
 265,
 266,
 267,
 268,
 271,
 272,
 273,
 278,
 281,
 284,
 285,
 286,
 287,
 288,
 291,
 293,
 294,
 297,
 299,
 301,
 302,
 303,
 304,
 305,
 307,
 314,
 315,
 318,
 320,
 334,
 339,
 341,
 342,
 347,
 349,
 355,
 358,
 360,
 361,
 362,
 366,
 367,
 371,
 374,
 377,
 378,
 380,
 384,
 393,
 394,
 396,
 397,
 398,
 400,
 401,
 402,
 406,
 409,
 410,
 412,
 414,
 415,
 417,
 418,
 419,
 420,
 421,
 422,
 423,
 426,
 427,
 431,
 432,
 433,
 434,
 436,
 437,
 439,
 440,
 442,
 446,
 449,
 461,
 462,
 464,
 466,
 